In [ ]:
import os
import cv2
import gradio as gr
import torch
import numpy as np
from PIL import Image
from diffusers import StableDiffusionPipeline

MANNEQUIN_PATH = "mannequin.png"  # Make sure this file exists (transparent PNG)

def overlay_on_mannequin(generated_img: Image.Image, mannequin_path: str) -> Image.Image:
    """
    Overlays the generated PIL image onto the mannequin image using alpha blending.
    Returns a new PIL image with the design on the mannequin.
    """
    # Convert the generated PIL image to OpenCV BGRA
    img_cv = cv2.cvtColor(np.array(generated_img.convert("RGBA")), cv2.COLOR_RGBA2BGRA)

    # Load the mannequin image (with alpha channel if present)
    mannequin = cv2.imread(mannequin_path, cv2.IMREAD_UNCHANGED)
    if mannequin is None:
        print("Error: mannequin image not found or invalid.")
        return generated_img

    # Ensure mannequin also has an alpha channel
    if mannequin.shape[2] == 3:
        mannequin = cv2.cvtColor(mannequin, cv2.COLOR_BGR2BGRA)

    # Resize the generated design to match the mannequin dimensions
    h, w = mannequin.shape[:2]
    img_resized = cv2.resize(img_cv, (w, h))

    # Blend using the design's alpha channel
    alpha = img_resized[:, :, 3] / 255.0
    blended = mannequin.copy()
    for c in range(3):
        blended[:, :, c] = (alpha * img_resized[:, :, c] + (1 - alpha) * blended[:, :, c]).astype(np.uint8)

    # Convert back to PIL in RGB
    blended_rgb = cv2.cvtColor(blended, cv2.COLOR_BGRA2RGB)
    return Image.fromarray(blended_rgb)

def generate_design(
    color, clothing_type, style, bust, waist, length, num_images, guidance_scale
):
    """
    Generates images using Stable Diffusion based on user inputs,
    then overlays them on the mannequin and returns the final images.
    """
    # Build a descriptive prompt
    prompt = (
        f"A {color} {style} {clothing_type} on a mannequin, "
        f"with bust {bust} cm, waist {waist} cm, length {length} cm, "
        "photorealistic, studio lighting, high detail"
    )

    # Negative prompt to avoid undesired colors or items
    negative_prompt = (
        "blue, green, purple, multi-color, pants, random colors, dress, lowres, blurry"
    )

    print("Prompt:", prompt)
    print("Negative Prompt:", negative_prompt)

    # Load the Stable Diffusion pipeline
    model_id = "runwayml/stable-diffusion-v1-5"
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16
    )
    pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

    # Generate images
    results = pipe(
        prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=50,         # You can adjust steps for quality vs. speed
        guidance_scale=float(guidance_scale),
        num_images_per_prompt=int(num_images),
    )

    # Overlay each generated image on the mannequin
    final_images = []
    for img in results.images:
        overlaid = overlay_on_mannequin(img, MANNEQUIN_PATH)
        final_images.append(overlaid)

    return final_images

# Create a Gradio interface
iface = gr.Interface(
    fn=generate_design,
    inputs=[
        gr.Textbox(label="Color", value="red", placeholder="e.g. bright red"),
        gr.Textbox(label="Clothing Type", value="blouse", placeholder="e.g. blouse, shirt, top"),
        gr.Textbox(label="Style (e.g., V-neck)", value="V-neck", placeholder="e.g. V-neck, short-sleeve"),
        gr.Number(label="Bust (cm)", value=34),
        gr.Number(label="Waist (cm)", value=28),
        gr.Number(label="Length (cm)", value=40),
        gr.Number(label="Number of Images", value=10),
        gr.Number(label="Guidance Scale (CFG)", value=7.5),
    ],
    outputs=gr.Gallery(label="Generated Designs", columns=2, height="auto"),
    title="Boutique Design Demo using Stable Diffusion",
    description=(
        "Generate design images using Stable Diffusion and overlay them on a mannequin.\n"
        "Adjust the parameters for color, style, measurements, etc. The numeric fields are interpreted as text.\n"
        "Use guidance scale to control how strongly the prompt is followed (higher = more strict)."
    ),
)
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a5875f3ddb82f75451.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install diffusers transformers accelerate --quiet
!pip install ipywidgets --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.0 MB/s eta 0:00:00


In [ ]:
!pip install flask pyngrok


In [ ]:
!pip install flask flask-ngrok


In [ ]:
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import io
import numpy as np
import cv2

upload_widget = widgets.FileUpload(accept="image/*", multiple=False)
color_widget = widgets.Text(value="red", description="Color:")
type_widget = widgets.Text(value="blouse", description="Type:")
style_widget = widgets.Text(value="V-neck", description="Style:")
bust_widget = widgets.IntSlider(value=34, min=30, max=50, description="Bust:")
waist_widget = widgets.IntSlider(value=28, min=20, max=50, description="Waist:")
length_widget = widgets.IntSlider(value=40, min=30, max=100, description="Length:")
guidance_widget = widgets.FloatSlider(value=7.5, min=3.0, max=15.0, step=0.5, description="CFG:")
generate_button = widgets.Button(description="Generate Design")

display(widgets.VBox([
    upload_widget, color_widget, type_widget, style_widget,
    bust_widget, waist_widget, length_widget, guidance_widget, generate_button
]))


NameError: name 'pipe' is not defined

In [ ]:
def overlay_on_human(human_img: Image.Image, generated_img: Image.Image):
    design_cv = cv2.cvtColor(np.array(generated_img.convert("RGBA")), cv2.COLOR_RGBA2BGRA)
    human_cv = cv2.cvtColor(np.array(human_img.convert("RGBA")), cv2.COLOR_RGBA2BGRA)
    h, w = human_cv.shape[:2]
    design_resized = cv2.resize(design_cv, (w, h))
    alpha = design_resized[:, :, 3] / 255.0
    blended = human_cv.copy()
    for c in range(3):
        blended[:, :, c] = (alpha * design_resized[:, :, c] + (1 - alpha) * blended[:, :, c]).astype(np.uint8)
    return Image.fromarray(cv2.cvtColor(blended, cv2.COLOR_BGRA2RGB))


In [ ]:
from IPython.display import display

def on_generate_click(b):
    clear_output(wait=True)

    if not upload_widget.value:
        print("Please upload an image.")
        return

    # Load uploaded image
    image_data = list(upload_widget.value.values())[0]['content']
    human_img = Image.open(io.BytesIO(image_data)).convert("RGBA")

    # Prepare prompt
    prompt = (
        f"A {color_widget.value} {style_widget.value} {type_widget.value}, "
        f"bust {bust_widget.value}cm, waist {waist_widget.value}cm, length {length_widget.value}cm, "
        "photorealistic, high detail, studio lighting"
    )
    negative_prompt = "nsfw, nude, naked, revealing, lowres, blurry"

    # Generate image
    result = pipe(prompt, negative_prompt=negative_prompt, guidance_scale=guidance_widget.value)
    generated_img = result.images[0]

    # Overlay
    final_img = overlay_on_human(human_img, generated_img)

    # Re-show UI and Output
    display(widgets.VBox([
        upload_widget, color_widget, type_widget, style_widget,
        bust_widget, waist_widget, length_widget, guidance_widget, generate_button
    ]))
    print("Prompt:", prompt)
    display(final_img)

generate_button.on_click(on_generate_click)


In [ ]:
!pip install diffusers transformers accelerate panel


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

KeyboardInterrupt: 

In [ ]:
import panel as pn
pn.extension()


In [ ]:
from PIL import Image
import torch
import numpy as np
import cv2
from io import BytesIO
from diffusers import StableDiffusionPipeline

# Load SD model
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

# UI widgets
upload_widget = pn.widgets.FileInput(accept="image/*", width=300)
prompt_widget = pn.widgets.TextInput(name="Prompt", placeholder="e.g. red V-neck blouse", width=300)
guidance_slider = pn.widgets.FloatSlider(name="Guidance Scale", start=3.0, end=15.0, step=0.5, value=7.5)
generate_button = pn.widgets.Button(name="Generate Design", button_type="primary", width=200)
output = pn.pane.PNG(height=512)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
def overlay_on_human(human_img: Image.Image, design_img: Image.Image) -> Image.Image:
    design_cv = cv2.cvtColor(np.array(design_img.convert("RGBA")), cv2.COLOR_RGBA2BGRA)
    human_cv = np.array(human_img.convert("RGBA"))  # ✅ fixed here

    h, w = human_cv.shape[:2]
    design_resized = cv2.resize(design_cv, (w, h))
    alpha = design_resized[:, :, 3] / 255.0
    blended = human_cv.copy()

    for c in range(3):
        blended[:, :, c] = (alpha * design_resized[:, :, c] + (1 - alpha) * blended[:, :, c]).astype(np.uint8)

    return Image.fromarray(cv2.cvtColor(blended, cv2.COLOR_RGBA2RGB))


def on_generate_click(event):
    if not upload_widget.value:
        output.object = None
        return

    # Load uploaded human image
    human_img = Image.open(BytesIO(upload_widget.value)).convert("RGBA")

    # Generate design image
    prompt = prompt_widget.value
    guidance = guidance_slider.value

    result = pipe(prompt, negative_prompt="nude, blurry, nsfw, lowres", guidance_scale=guidance)
    design_img = result.images[0]

    # Overlay
    final_img = overlay_on_human(human_img, design_img)
    buffer = BytesIO()
    final_img.save(buffer, format='PNG')
    output.object = buffer.getvalue()

generate_button.on_click(on_generate_click)


Watcher(inst=Button(button_type='primary', clicks=1, name='Generate Design', width=200), cls=<class 'panel.widgets.button.Button'>, fn=<function on_generate_click at 0x7a9ffc53b240>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [ ]:
app_ui = pn.Column(
    "## 👗 AI Clothing Design on Human Image (No Gradio)",
    pn.Row(upload_widget, prompt_widget),
    guidance_slider,
    generate_button,
    output
)
app_ui


Column
    [0] Markdown(str)
    [1] Row
        [0] FileInput(accept='image/*', filename='womanmodel.jpg', mime_type='image/jpeg', value=b'\xff\xd8\xff\xe0\x00\x10..., width=300)
        [1] TextInput(name='Prompt', placeholder='e.g. red V-neck blouse', value='red v neck blouse o..., value_input='red v neck blouse o...)
    [2] FloatSlider(end=15.0, name='Guidance Scale', start=3.0, step=0.5, value=7.5)
    [3] Button(button_type='primary', clicks=1, name='Generate Design', width=200)
    [4] PNG(None, height=512)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]